In [11]:
import requests
import json
import argparse
import sys

In [12]:
URL = 'https://api.data.caida.org/as2org/v1'
ASRANK_URL = 'https://api.asrank.caida.org/v2'

In [22]:
def org(targetASN):
  """
  Download the organization for a target AS
  """
  # Get orgid given asn
  url_built = f"{URL}/asns/{targetASN}/"
  org = None
  orgId = 0

  try:
    response = getJsonResponse(url_built)
    orgId = response["data"][0]["orgId"]

  except Exception as err:
    print(url_built)
    print(err)

  # Get org given orgid
  url_built = f"{URL}/orgs/{orgId}"
  try:
    response = getJsonResponse(url_built)
    org = response["data"][0]
  except Exception as err:
    print(url_built)
    print(err)
  if org is not None:
    return org["orgName"]

In [15]:
asns = 'as_rank_dataset.json'
with open(asns, 'r') as f:
    asns = json.load(f)

In [ ]:
orgs = set()
for asn in asns:
    organization = org(asn)
    print(organization)
    orgs.add(organization)

In [26]:
orgs = list(orgs)

In [ ]:
orgs

In [13]:
# Helper method
def getJsonResponse(URL):
  response = requests.get(URL)
  return response.json()

In [14]:
def findASN(orgName):
  """
  Find ASN of largest AS of given org name
  """
  # Find org id associated with orgName
  try:
    url_built = f"{URL}/orgs"
    orgs = getJsonResponse(url_built)
    orgId = 0
    for org in orgs["data"]:
      if (org["orgName"] == orgName):
        orgId = org["orgId"]
  except Exception as err:
    print(url_built)
    print(err)

  asrank_org = getJsonResponse(f"{ASRANK_URL}/restful/organizations/{orgId}")
  return asrank_org["data"]["organization"]["members"]["asns"]["edges"][0]["node"]["asn"]

In [40]:
def largestAS(targetOrgName):
  """
  Download largest AS with a given organization name
  """
  largestASN = 0
  largestAS = None

  try:
    largestASN = findASN(targetOrgName)
  except Exception as err:
    print(err)
  if largestASN:
    url_built = f"{URL}/asns/{largestASN}"
    try:
      response = getJsonResponse(url_built)
      largestAS = response["data"][0]
    except Exception as err:
      print(url_built)
      print(err)
    asn = largestAS["asn"]
    return asn

In [34]:
orgs= orgs[1:len(orgs)]

In [ ]:
org_largest_asn = dict()
for organization in orgs:
    if organization:
      print(organization)
      largest_asn = largestAS(organization)
      if largest_asn:
        print(largest_asn)
        org_largest_asn[organization] = largest_asn

In [50]:
# Replace 'output.json' with your desired file name
with open('org_largest_asn.json', 'w') as f:
    json.dump(org_largest_asn, f, indent=4)

In [ ]:
org_largest_asn

In [9]:
def NumOfASNsInOrg(orgName):
  """
  Find number of ASNs in an organization
  """
  # Find org id associated with orgName
  try:
    url_built = f"{URL}/orgs"
    orgs = getJsonResponse(url_built)
    orgId = 0
    orgMembers = None
    for org in orgs["data"]:
      if (org["orgName"] == orgName):
        orgId = org["orgId"]
        orgMembers = org["members"]
  except Exception as err:
    print(url_built)
    print(err)

  return len(orgMembers)

In [54]:
orgs_dataset = dict()
for org in org_largest_asn:
  orgs_dataset[org] = {
      'largest_asn': org_largest_asn[org],
      'num_asns': NumOfASNsInOrg(org)
  }

In [55]:
# Replace 'output.json' with your desired file name
with open('orgs_dataset.json', 'w') as f:
    json.dump(orgs_dataset, f, indent=4)